The following exercises are meant to be solved by gathering the bash commands incrimentally in two scripts, one for ex 1.* the other for ex 2.* 

### Ex 1

1\.a Make a new directory called `students` in your home. Download a csv file with the list of students of this lab from [here](https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv) (use the `wget` command) and copy that to `students`. First check whether the file is already there

In [ ]:
mkdir students1
cd students1
dir students1
wget https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv

1\.b Make two new files, one containing the students belonging to PoD, the other to Physics.

In [ ]:
touch PoD
grep -i "PoD" LCP_22-23_students.csv >> PoD
touch Physics
grep -i "Physics" LCP_22-23_students.csv >> Physics

1\.c For each letter of the alphabet, count the number of students whose surname starts with that letter. 

In [1]:

#!/bin/bash
declare -A char_counts
for i in {A..Z}; do
  counter=0
  while IFS=, read -r A _; do
    if [[ ${A:0:1} == $i ]]; then
      ((counter=counter+1))
    fi
  done < <(tail -n +2 LCP_22-23_students.csv | cut -d ',' -f 1)
  char_counts["$i"]=$counter
done

SyntaxError: invalid decimal literal (1575255014.py, line 9)

1\.d Find out which is the letter with most counts.

In [ ]:
echo "" > ct
touch maxval
max=0
for i in {A..Z}; do
  echo "$i : ${char_counts[$i]}" >> ct
  if [[ "${char_counts[$i]}" -gt "$max" ]]; then
    max=${char_counts[$i]}
  fi
done
echo "$max" > maxval

1\.e Assume an obvious numbering of the students in the file (first line is 1, second line is 2, etc.), group students "modulo 18", i.e. 1,19,37,.. 2,20,38,.. etc. and put each group in a separate file  

In [ ]:
line_number=0
while IFS= read -r line; do
  j=$(( line_number % 18 ))
  output_file="Mod18_group_${j}.csv"
  echo "$line" >> "$output_file"
  ((line_number++))
done < <(tail -n+2 LCP_22-23_students.csv)


### Ex 2

2.a Make a copy of the file `data.csv` removing the metadata and the commas between numbers; call it `data.txt`

In [ ]:

#!/bin/bash
touch tmp
tail -n+4 data.csv > tmp
awk 'NR > 1 {for (i=1; i<=NF; i++) gsub(/,/, " ", $i)}1' tmp > data.txt
rm tmp
counter=0
greater_count=0
smaller_count=0




2\.b How many even numbers are there?

2\.c Distinguish the entries on the basis of `sqrt(X^2 + Y^2 + Z^2)` is greater or smaller than `100*sqrt(3)/2`. Count the entries of each of the two groups 

In [ ]:
while IFS= read -r line; do
  IFS=" " read -ra fields <<< "$line"
    X=${fields[0]}
    Y=${fields[1]}
    Z=${fields[2]}
    value=$(echo "scale=10; sqrt($X^2 + $Y^2 + $Z^2)" | bc)
    threshold=$(echo "scale=10; 100*sqrt(3)/2" | bc)
    if (( $(echo "$value > $threshold" | bc -l) )); then
    ((greater_count++))
    else
    ((smaller_count++))
    fi
  for field in "${fields[@]}"; do
    if ((field % 2 == 0)); then
      ((counter++))
    fi
  done
done < <(tail -n +2 data.txt)
echo "$counter"
echo "Entries with sqrt(X^2 + Y^2 + Z^2) > 100*sqrt(3)/2: $greater_count"
echo "Entries with sqrt(X^2 + Y^2 + Z^2) <= 100*sqrt(3)/2: $smaller_count"

2\.d Make `n` copies of data.txt (with `n` an input parameter of the script), where the i-th copy has all the numbers divided by i (with `1<=i<=n`).

In [ ]:


if [ $# -ne 1 ]; then
  echo "Usage: $0 <n>"
  exit 1
fi
n=$1
if [ ! -f data.txt ]; then
  echo "data.txt not found."
  exit 1
fi
for ((i = 1; i <= n; i++)); do
  output_file="data_divided_by_${i}.txt"
  awk -v factor="$i" 'NR == 1 {print; next} {for (j = 1; j <= NF; j++) $j /= factor; print}' data.txt > "$output_file"
  echo "Created $output_file"
done